# Make an SED!

### Requirements
- astropy-3.0.2
- bokeh-0.12.6

In [72]:
# Imports
from SEDkit import sed, spectrum, synphot
import astropy.units as q
import numpy as np
from pkg_resources import resource_filename
from bokeh.io import output_notebook, show
output_notebook()

Loading BokehJS ...

Let's create a new SED for an M-dwarf. The software assumes a field age if no `age` argument is given.

In [73]:
s = sed.SED(parallax=(1.7474*q.mas, 0.0501*q.mas), spectral_type='M0V', name='2MASS 261832886')

Make this handle asymmetric uncertainties!
Setting age to (<Quantity 6.0 Gyr>, <Quantity 4.0 Gyr>)


Now let's add some photometry and take a look at how it's stored!

In [74]:
# You can pass an ascii file of photometry with the column names 
# 'band', 'magnitude', and 'uncertainty' like so...
s.add_photometry_file(resource_filename('SEDkit', 'data/M0V_photometry.txt'))

# Or you can add individual measurements like so...
s.add_photometry('Kepler.K', 16.132, 0.052)
s.add_photometry('WISE.W4', 11.1, None)

# Take a look
print(s.photometry)

  band           eff         app_magnitude ...      abs_flux_unc      bandpass
                  um                       ... erg / (Angstrom cm2 s)         
-------- ------------------- ------------- ... ---------------------- --------
  SDSS.u 0.35565239687948447        20.486 ...                    0.0   SDSS.u
  SDSS.g  0.4702495279331733        17.779 ...                    0.0   SDSS.g
  SDSS.r  0.6175578881059804        16.431 ...                    0.0   SDSS.r
Kepler.K  0.6303355872125808        16.132 ...                    0.0 Kepler.K
  SDSS.i  0.7489976846298569        15.843 ...                    0.0   SDSS.i
  SDSS.z   0.894670956505578        15.514 ...                    0.0   SDSS.z
 2MASS.J  1.2393093155660664        14.391 ...                    0.0  2MASS.J
 2MASS.H  1.6494947091246095        13.677 ...                    0.0  2MASS.H
2MASS.Ks   2.163860596453316        13.535 ...                    0.0 2MASS.Ks
 WISE.W1  3.3897048577485647         13.55 ...      

Spectra aren't necessary to build an SED but let's add some for fun.

In [75]:
# Add a spectrum from a file, specifying the astropy units
file = resource_filename('SEDkit', 'data/M0V_spectrum.txt')
s.add_spectrum_file(file, wave_units=q.um, flux_units=q.W/q.m**2/q.um)

# Alternatively, you can add the spectrum manually
"""
wave, flux, unc = np.genfromtxt(file, unpack=True)
s.add_spectrum(wave*q.um, flux*q.W/q.m**2/q.um, unc*q.W/q.m**2/q.um)
"""

# Take a look
show(s.spectra[0].plot)

Now that our SED has all the data it needs, we can call the `results` method, which constructs the SED and caluclates the fundamental parameters.

If no `radius` is given, one will be estimated from model isochrones using the calculated `Lbol` and assumed `age`.

In [76]:
print(s.results)

Setting radius to (<Quantity 0.8786209573091851 solRad>, <Quantity 0.06782337214316517 solRad>)
    param          value        unc       units    
------------- --------------- ------- -------------
         name 2MASS 261832886      --            --
          age             6.0     4.0           Gyr
     distance          572.28   16.41            pc
     parallax          1.7474  0.0501           mas
       radius           0.879   0.068        solRad
spectral_type            60.0     0.5            --
   membership              --      --            --
         fbol             0.0     0.0 erg / (cm2 s)
         mbol          16.041   0.003            --
         Lbol        3.84e+32 2.2e+31       erg / s
     Lbol_sun          -0.999   0.025            --
         Mbol           7.253   0.031            --
         logg             4.5    0.07            --
         mass            0.89     0.0       solMass
         Teff            3464     142             K


In [77]:
fig = s.plot()

But that radius seems too small. Let's change it...

In [78]:
s.radius = 0.6*q.Rsun, 0.05*q.Rsun

Setting radius to (<Quantity 0.6 solRad>, <Quantity 0.05 solRad>)


In [79]:
print(s.results)

    param          value        unc       units    
------------- --------------- ------- -------------
         name 2MASS 261832886      --            --
          age             6.0     4.0           Gyr
     distance          572.28   16.41            pc
     parallax          1.7474  0.0501           mas
       radius             0.6    0.05        solRad
spectral_type            60.0     0.5            --
   membership              --      --            --
         fbol             0.0     0.0 erg / (cm2 s)
         mbol          16.041   0.003            --
         Lbol        3.84e+32 2.2e+31       erg / s
     Lbol_sun          -0.999   0.025            --
         Mbol           7.253   0.031            --
         logg             4.5    0.07            --
         mass            0.89     0.0       solMass
         Teff            4192     184             K


... which is right since if we assume a larger radius with the same Lbol, the Teff should be lower.